# 03. Object recognition of CIFAR10 dataset


---
## Purpose
Carry out object recognition of the CIFAR10 dataset. The structure of this program is similar to the MNIST character recognition program, so refer to that tutorial for a basic explanation. This page describes differences with the MNIST character recognition program.

Compute neural network operations by using the GPU．


## Preparations

### Confirm and change Google Colaboratory settings

In this tutorial, we use PyTorch to implement a neural network and carry out training and evaluation. **To process operations using the GPU, go to the menu bar at the top of screen and choose Runtime -> Change runtime type -> Hardware accelerator -> GPU.**


## Dataset

### CIFAR10 dataset

We use the CIFAR10 dataset for object recognition in this tutorial. The CIFAR10 dataset is composed of images in 10 different classes representing airplanes, dogs, etc.

![CIFAR10_sample.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/176458/b6b43478-c85f-9211-7bc6-227d9b387af5.png)

## Import modules

First, import the necessary modules.

### Confirm GPU settings

Confirm computation using GPU is enabled.
If `Use CUDA: True` is displayed, it is possible to use the GPU to perform computation in PyTorch. **If Use CUDA: False is displayed, start from the procedures given in “Confirm and change Google Colaboratory settings” above and change the settings. Then import the modules again.**



In [ ]:
# import modules
from time import time
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

import torchsummary

# confirm GPU settings
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## Read and confirm dataset

Load the training data (CIFAR10 dataset)．

Confirm the size of the loaded data. The training data size is 50,000 images. The data dimensions of each image are 3x32x32. This indicates a color image of 32x32 pixels.

In [ ]:
train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(train_data)
print(test_data)

### Display CIFAR10 dataset

Display the images in the CIFAR10 dataset. Here we use a program that uses matplotlib to display multiple images.


In [ ]:
import matplotlib.pyplot as plt

cols = 10
rows = 2

plt.clf()
fig = plt.figure(figsize=(14, 4.8))
for r in range(rows):
    for c in range(cols):
        ax = fig.add_subplot(r+1, cols, c+1)
        ax.imshow(train_data[c+r*cols][0].permute(1, 2, 0))
        ax.set_axis_off()
plt.show()

## Define neural network model

Define the convolutional neural network.

The network in this tutorial consists of two convolutional layers and three fully connected layers.

The first convolutional layer has 1 input channel, 16 output feature maps, and a 3x3 convolution filter. The second convolutional layer has 16 input channels, 32 output feature maps, and convolution filter that also has a size of 3x3. The first fully connected layer has `7*7*32` input units and 1024 output units. The next fully connected layer has 1024 input units and 1024 output units. The output layer has 1024 input units and 10 output units. For the activation function, we define a sigmoid function in `self.act`. In addition, we define self.pool to carry out pooling. For this example, we use max pooling. We define the composition of each layer using the `__init__` function.


The `forward` function describes how to connect and process the defined layers. The `forward` function’s parameter `x` represents the input data. This parameter’s argument is inputted to `conv1` defined by the `__init__` function. The output is passed to the activation function `self.act`. The output of that function is passed to `self.pool`. The result of pooling is outputted as `h`. The second convolutional layer is also processed using the same procedures.

After convolution is applied to the feature map, the map is inputted to the fully connected layers. Identification results are outputted. First, the shape (channel x height x width) of the feature map obtained by convolution is converted to a one-dimensional array. Here we manipulate array `h` by using `view()`. We obtain the first dimension of the size of h with the first argument, `h.size()[0]`, and specify it as the size of the first dimension of the array after conversion. The second argument, `-1`, specifies an arbitrary size. Doing so transforms `h` to the shape (number of batches x arbitrary length of data). Finally, the class scores are returned by sequentially inputting the converted `h` to the fully connected layers and the activation function.


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.l1 = nn.Linear(8 * 8 * 32, 1024)
        self.l2 = nn.Linear(1024, 1024)
        self.l3 = nn.Linear(1024, 10)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        h = self.pool(self.act(self.conv1(x)))
        h = self.pool(self.act(self.conv2(h)))
        h = h.view(h.size()[0], -1)
        h = self.act(self.l1(h))
        h = self.act(self.l2(h))
        h = self.l3(h)
        return h

## Create neural network


Create the neural network defined by the program above.

Call the `CNN` class to define the neural network model. If using the GPU （`use_cuda == True`）, the network model is placed in GPU memory. This makes it possible to perform operations using the GPU.
We use stochastic gradient descent with momentum (SGD with momentum) as the optimization technique when training. We pass 0.01 as the argument of the learning rate parameter and 0.9 as the argument of the momentum parameter.


Finally, `torchsummary.summary()` is used to display detailed information about the defined network.


In [ ]:
model = CNN()
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# display the detailed information about the defined network
torchsummary.summary(model, (3, 32, 32))

## Training
Carry out training by using the loaded CIFAR10 dataset and the created neural network.

We set the data size for calculating errors for one pass (mini-batch size) as 64 and the number of training epochs as 10. 

Next, we define the data loader. The data loader uses the training dataset (`train_data`) that was loaded above and creates an object that reads the data in the mini-batch size as specified by the assignment statement below. For this training, we set `shuffle=True` to specify that the data is to be read randomly each time.

Next, we set the error function. Because we are dealing with a classification problem here, we define `criterion` to be `CrossEntropyLoss` to calculate cross entropy error.

Begin training.

For each update, the data to be learned and the teacher data are given the names `image` and `label`, respectively. The training model is given an image and obtains the probability y for each class. The error between each class’s probability y and the teacher label is calculated by `criterion`. The recognition accuracy is also calculated. The error is then backpropagated by the backward function to update the neural network.


In [ ]:
# set the mini-batch size and training epocs
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# define data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# set the error (loss) function
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# switch the network configuration into training mode
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)
        
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_loader),
                                                                                 time() - start))

## Testing
Evaluate by using the trained network model on the testing data.

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## Problems

### 1. Change the neural network structure and confirm the change in recognition accuracy.

**Hint: The following items can change the neural network structure.**
* The number of units in intermediate layers, convolution kernel size of convolution
* Number of layers
* The activation function
  * For example, `nn.Tanh()` or `nn.ReLU()`, `nn.LeakyReLU()`, etc.
  * Other activation functions that can be used in PyTorch are summarized on [this page](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity).


### 2. Change training settings and confirm the change in recognition accuracy.

**Hint: The following settings that can be changed in the program**
* Mini-batch size
* Number of training cycles (number of epochs)
* Learning rate
* Optimization method
  * Choices include `torch.optim.Adagrad()` and `torch.optim.Adam()`．
  * Optimization methods that can be used in PyTorch are summarized on [this page](https://pytorch.org/docs/stable/optim.html#algorithms).
